# Advanced SQL

In [2]:
import sqlite3
conn = sqlite3.connect('factbook.db')
schema = conn.execute('pragma table_info(facts)').fetchall()
for items in schema:
    print items

(0, u'id', u'INTEGER', 1, None, 1)
(1, u'code', u'varchar(255)', 1, None, 0)
(2, u'name', u'varchar(255)', 1, None, 0)
(3, u'area', u'integer', 0, None, 0)
(4, u'area_land', u'integer', 0, None, 0)
(5, u'area_water', u'integer', 0, None, 0)
(6, u'population', u'integer', 0, None, 0)
(7, u'population_growth', u'float', 0, None, 0)
(8, u'birth_rate', u'float', 0, None, 0)
(9, u'death_rate', u'float', 0, None, 0)
(10, u'migration_rate', u'float', 0, None, 0)
(11, u'created_at', u'datetime', 0, None, 0)
(12, u'updated_at', u'datetime', 0, None, 0)


# Query plan

Explore the high level query plan that would be performed
- scan table <> : scan the entire table

In [3]:
conn = sqlite3.connect("factbook.db")
query_plan_one = conn.execute('explain query plan select * from facts where area > 40000').fetchall()
query_plan_two = conn.execute('explain query plan select area from facts where area > 40000').fetchall()
query_plan_three = conn.execute('explain query plan select * from facts where name = "Czech Republic"').fetchall()

print(query_plan_one)
print(query_plan_two)
print(query_plan_three)

[(0, 0, 0, u'SCAN TABLE facts')]
[(0, 0, 0, u'SCAN TABLE facts')]
[(0, 0, 0, u'SCAN TABLE facts')]


The above three queries rae the same -- they all scan the entire table.

This is because we set the id column as the primary key and SQLite uses this column to order the records in the database file. 

Unless we specify the id key, SQLite does not know how to search for a specific row.

## Searching with specific index

- order logN vs order N

In [4]:
# where id = # allows Order(logN) binary search
query_plan_four=conn.execute('explain query plan select * from facts where id=20').fetchall()
print(query_plan_four)

[(0, 0, 0, u'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]


# Indexing
Creating a separate table that's optimized for lookups by a different column instead of by the id.
![index](indexing.png)

**Pros:** 
- Can speed up db querying process from order N to order logN

**Cons:**
- Need memory/space to create table
- Need to manually update/insert/delete new index table since it was created as a post-process

## Creating an index
CREATE INDEX IF NOT EXISTS index_name ON table_name(column_name);


In [5]:
# Query using non-primary key column
query_plan_six = conn.execute('explain query plan select * from facts where population > 10000').fetchall()
print(query_plan_six)

# Create index and query based on index
conn.execute('CREATE INDEX pop_idx ON facts(population)')
query_plan_seven = conn.execute('explain query plan select * from facts where population > 10000').fetchall()
print(query_plan_seven)

[(0, 0, 0, u'SCAN TABLE facts')]
[(0, 0, 0, u'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


In [6]:
rows=conn.execute('pragma table_info(facts)').fetchall()
for row in rows:
    print row

(0, u'id', u'INTEGER', 1, None, 1)
(1, u'code', u'varchar(255)', 1, None, 0)
(2, u'name', u'varchar(255)', 1, None, 0)
(3, u'area', u'integer', 0, None, 0)
(4, u'area_land', u'integer', 0, None, 0)
(5, u'area_water', u'integer', 0, None, 0)
(6, u'population', u'integer', 0, None, 0)
(7, u'population_growth', u'float', 0, None, 0)
(8, u'birth_rate', u'float', 0, None, 0)
(9, u'death_rate', u'float', 0, None, 0)
(10, u'migration_rate', u'float', 0, None, 0)
(11, u'created_at', u'datetime', 0, None, 0)
(12, u'updated_at', u'datetime', 0, None, 0)


## Using created index

In [ ]:
conn = sqlite3.connect("factbook.db")
india_index = conn.execute('select id from name_idx where name = "India"').fetchall()[0][0]
print(india_index)

first_query_plan = conn.execute('explain query plan select id from name_idx where name = "India"').fetchall()
print(first_query_plan)

india_row = conn.execute('select * from facts where id = ?',(india_index,)).fetchall()
print(india_row)

second_query_plan = conn.execute('explain query plan select * from facts where id = ?',(india_index,)).fetchall()
print(second_query_plan)